<a href="https://colab.research.google.com/github/MUbarak123-56/qiskit-work/blob/master/quantum_applications/linear_systems_hhl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
from linear_solvers import NumPyLinearSolver, HHL
matrix = np.array([ [1, -1/3], [-1/3, 1] ])
vector = np.array([1, 0])
naive_hhl_solution = HHL().solve(matrix, vector)

In [3]:
classical_solution = NumPyLinearSolver().solve(matrix,
                                               vector/np.linalg.norm(vector))

In [4]:
classical_solution

In [5]:
from linear_solvers.matrices.tridiagonal_toeplitz import TridiagonalToeplitz
tridi_matrix = TridiagonalToeplitz(1, 1, -1 / 3)
tridi_solution = HHL().solve(tridi_matrix, vector)

In [6]:
print('classical state:', classical_solution.state)

classical state: [1.125 0.375]


In [7]:
print('naive state:')
print(naive_hhl_solution.state)
print('tridiagonal state:')
print(tridi_solution.state)

naive state:
      ┌─────────────┐┌──────┐        ┌─────────┐
  q4: ┤ circuit-121 ├┤3     ├────────┤3        ├
      └─────────────┘│      │┌──────┐│         │
q5_0: ───────────────┤0     ├┤2     ├┤0        ├
                     │  QPE ││      ││  QPE_dg │
q5_1: ───────────────┤1     ├┤1     ├┤1        ├
                     │      ││  1/x ││         │
q5_2: ───────────────┤2     ├┤0     ├┤2        ├
                     └──────┘│      │└─────────┘
  q6: ───────────────────────┤3     ├───────────
                             └──────┘           
tridiagonal state:
       ┌─────────────┐┌──────┐        ┌─────────┐
  q26: ┤ circuit-334 ├┤3     ├────────┤3        ├
       └─────────────┘│      │┌──────┐│         │
q27_0: ───────────────┤0     ├┤2     ├┤0        ├
                      │  QPE ││      ││  QPE_dg │
q27_1: ───────────────┤1     ├┤1     ├┤1        ├
                      │      ││  1/x ││         │
q27_2: ───────────────┤2     ├┤0     ├┤2        ├
                      └──────

In [8]:
print('classical Euclidean norm:', classical_solution.euclidean_norm)
print('naive Euclidean norm:', naive_hhl_solution.euclidean_norm)
print('tridiagonal Euclidean norm:', tridi_solution.euclidean_norm)

classical Euclidean norm: 1.1858541225631423
naive Euclidean norm: 1.1858541225631376
tridiagonal Euclidean norm: 1.1858541225631356


In [9]:
from qiskit.quantum_info import Statevector

naive_sv = Statevector(naive_hhl_solution.state).data
tridi_sv = Statevector(tridi_solution.state).data

# Extract vector components; 10000(bin) == 16 & 10001(bin) == 17
naive_full_vector = np.array([naive_sv[16], naive_sv[17] ])
tridi_full_vector = np.array([tridi_sv[16], tridi_sv[17] ])

print('naive raw solution vector:', naive_full_vector)
print('tridi raw solution vector:', tridi_full_vector)

naive raw solution vector: [0.75+3.01063111e-16j 0.25+3.36951198e-17j]
tridi raw solution vector: [0.75-7.11448145e-17j 0.25+3.64923833e-16j]


In [10]:
def get_solution_vector(solution):
    """Extracts and normalizes simulated state vector
    from LinearSolverResult."""
    solution_vector = Statevector(solution.state).data[16:18].real
    norm = solution.euclidean_norm
    return norm * solution_vector / np.linalg.norm(solution_vector)

print('full naive solution vector:', get_solution_vector(naive_hhl_solution))
print('full tridi solution vector:', get_solution_vector(tridi_solution))
print('classical state:', classical_solution.state)

full naive solution vector: [1.125 0.375]
full tridi solution vector: [1.125 0.375]
classical state: [1.125 0.375]


In [11]:
from scipy.sparse import diags

NUM_QUBITS = 2
MATRIX_SIZE = 2 ** NUM_QUBITS
# entries of the tridiagonal Toeplitz symmetric matrix
a = 1
b = -1/3

matrix = diags([b, a, b],
               [-1, 0, 1],
               shape=(MATRIX_SIZE, MATRIX_SIZE)).toarray()

vector = np.array([1] + [0]*(MATRIX_SIZE - 1))
# run the algorithms
classical_solution = NumPyLinearSolver(
                        ).solve(matrix, vector / np.linalg.norm(vector))
naive_hhl_solution = HHL().solve(matrix, vector)
tridi_matrix = TridiagonalToeplitz(NUM_QUBITS, a, b)
tridi_solution = HHL().solve(tridi_matrix, vector)

print('classical euclidean norm:', classical_solution.euclidean_norm)
print('naive euclidean norm:', naive_hhl_solution.euclidean_norm)
print('tridiagonal euclidean norm:', tridi_solution.euclidean_norm)

classical euclidean norm: 1.237833351044751
naive euclidean norm: 1.2099806231118977
tridiagonal euclidean norm: 1.209457721870527


In [12]:
matrix

array([[ 1.        , -0.33333333,  0.        ,  0.        ],
       [-0.33333333,  1.        , -0.33333333,  0.        ],
       [ 0.        , -0.33333333,  1.        , -0.33333333],
       [ 0.        ,  0.        , -0.33333333,  1.        ]])

In [13]:
print('naive state:')
print(naive_hhl_solution.state)
print('tridiagonal state:')
print(tridi_solution.state)

naive state:
        ┌──────────────┐┌──────┐        ┌─────────┐
q107_0: ┤0             ├┤4     ├────────┤4        ├
        │  circuit-669 ││      │        │         │
q107_1: ┤1             ├┤5     ├────────┤5        ├
        └──────────────┘│      │┌──────┐│         │
q108_0: ────────────────┤0     ├┤3     ├┤0        ├
                        │  QPE ││      ││  QPE_dg │
q108_1: ────────────────┤1     ├┤2     ├┤1        ├
                        │      ││      ││         │
q108_2: ────────────────┤2     ├┤1 1/x ├┤2        ├
                        │      ││      ││         │
q108_3: ────────────────┤3     ├┤0     ├┤3        ├
                        └──────┘│      │└─────────┘
  q109: ────────────────────────┤4     ├───────────
                                └──────┘           
tridiagonal state:
        ┌───────────────┐┌──────┐        ┌─────────┐
q160_0: ┤0              ├┤4     ├────────┤4        ├
        │  circuit-1297 ││      │        │         │
q160_1: ┤1              ├┤5  

In [14]:
from qiskit import transpile

MAX_QUBITS = 4
a = 1
b = -1/3

i = 1
# calculate the circuit depths for different number of qubits to compare the use
# of resources (WARNING: This will take a while to execute)
naive_depths = []
tridi_depths = []
for n_qubits in range(1, MAX_QUBITS+1):
    matrix = diags([b, a, b],
                   [-1, 0, 1],
                   shape=(2**n_qubits, 2**n_qubits)).toarray()
    vector = np.array([1] + [0]*(2**n_qubits -1))

    naive_hhl_solution = HHL().solve(matrix, vector)
    tridi_matrix = TridiagonalToeplitz(n_qubits, a, b)
    tridi_solution = HHL().solve(tridi_matrix, vector)

    naive_qc = transpile(naive_hhl_solution.state,
                         basis_gates=['id', 'rz', 'sx', 'x', 'cx'])
    tridi_qc = transpile(tridi_solution.state,
                         basis_gates=['id', 'rz', 'sx', 'x', 'cx'])

    naive_depths.append(naive_qc.depth())
    tridi_depths.append(tridi_qc.depth())
    i +=1

In [15]:
sizes = [f"{2**n_qubits}×{2**n_qubits}"
         for n_qubits in range(1, MAX_QUBITS+1)]
columns = ['size of the system',
           'quantum_solution depth',
           'tridi_solution depth']
data = np.array([sizes, naive_depths, tridi_depths])
ROW_FORMAT ="{:>23}" * (len(columns) + 2)
for team, row in zip(columns, data):
    print(ROW_FORMAT.format(team, *row))

     size of the system                    2×2                    4×4                    8×8                  16×16
 quantum_solution depth                    334                   2562                  33141                 348396
   tridi_solution depth                    565                   5107                  14756                  46552


In [16]:
print('excess:',
      [naive_depths[i] - tridi_depths[i] for i in range(0, len(naive_depths))])

excess: [-231, -2545, 18385, 301844]


In [17]:
from linear_solvers.observables import AbsoluteAverage, MatrixFunctional

In [18]:
from qiskit import Aer

backend = Aer.get_backend('aer_simulator')
hhl = HHL(1e-3, quantum_instance=backend)

accurate_solution = hhl.solve(matrix, vector)
classical_solution = NumPyLinearSolver(
                    ).solve(matrix,
                            vector / np.linalg.norm(vector))

print(accurate_solution.euclidean_norm)
print(classical_solution.euclidean_norm)

1.2105959357182015
1.239912714366469
